# Import Modules

In [1]:
#%qtconsole
#%matplotlib inline

import matplotlib.pyplot as plt
from functools import partial
import numpy as np
import time 
import qcodes as qc
import os
from qcodes import Station, load_or_create_experiment, initialise_database, Measurement, load_by_run_spec, load_by_guid, initialise_or_create_database_at 
from qcodes.utils.dataset.doNd import do1d, do2d, plot

from qcodes.instrument.visa import VisaInstrument
from qcodes_contrib_drivers.drivers.StanfordResearchSystems.SIM928 import SIM928
from qcodes.instrument_drivers.stanford_research.SR830 import SR830
from qcodes.dataset.plotting import plot_dataset
import IPython.lib.backgroundjobs as bg
from plottr.apps import inspectr

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\manfra-lab\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\manfra-lab\.qcodes\logs\231017-19360-qcodes.log


# Connect to Instruments and Initialize Variables

In [2]:
#SIM Controller
sim=SIM928('Janissary','GPIB0::15::INSTR')
#Lockins SR830
lockIn=SR830('Gommorah','GPIB0::8::INSTR' )

Connected to: Stanford_Research_Systems SIM900 (serial:s/n130891, firmware:ver3.6) in 1.05s
Connected to: Stanford_Research_Systems SR830 (serial:s/n21560, firmware:ver1.01) in 0.09s


# Global Parameters for Instruments and Experiment

In [3]:
dataBasePath = 'C:\\Users\\manfra-lab\\Desktop\\Users\\Matt\\Data\\QcodesTest\\4K_Sample_MMDDYY.db'

In [4]:
exp_name = "QPC_Conductance"

In [16]:
meas_name = 'Gates4&3_SweepDown'

In [18]:
csvPath = "C:\\Users\\manfra-lab\\Desktop\\Users\\Matt\\Data\\QcodesTest\\placeHolder.csv"
sample = "8-3-22.1MM11A3"
#Module number on SIM controller
sim_port = 1
#Units are in volts
V_start = sim.get_voltage(sim_port)
V_end = 0.5
#Volt Step(Volts)
rampRate = 0.001
#Time Delay(seconds)
delay = 1
#Changes direction of ramp if ramping down to a voltage instead of up
if V_end < V_start:
    rampRate = rampRate *(-1)
#Acquires module name based on SIM port
name = sim.slot_names.get(sim_port, sim_port)
module = 'volt_{}'.format(name)

# Set Up Database for Data Collection

In [7]:
db_file_path = os.path.join(os.getcwd(), dataBasePath)
initialise_or_create_database_at(db_file_path)

# Set Up Live Plot

In [8]:
#Opens windows that will plot results as they are collected
#Make sure to turn on refresh rate once window opens or nothing will happen
jobs = bg.BackgroundJobManager()
jobs.new(inspectr.main, db_file_path)

<BackgroundJob #0: <function main at 0x0000022291896700>>

# Setup Station and Measurement Parameters

In [9]:
#Handles equipment
station = qc.Station()
station.add_component(sim)
station.add_component(lockIn)

'Gommorah'

In [17]:
#Manages independent and dependent variables
experiment = load_or_create_experiment(
    experiment_name=exp_name,
    sample_name=sample
)
context_meas = Measurement(exp=experiment, station=station, name=meas_name)
context_meas.write_period = 0.1

context_meas.register_parameter(sim.parameters[module])
context_meas.register_parameter(lockIn.X, setpoints=(sim.parameters[module],))

# Sweep DC Voltage and Measure Results

In [19]:
with context_meas.run() as datasaver:
    for v_set in np.arange(V_start, V_end+rampRate, rampRate):
        sim.set_voltage(sim_port, v_set)
        get_x = lockIn.X.get()
        datasaver.add_result((sim.parameters[module], v_set),
                             (lockIn.X, get_x))        
        time.sleep(delay)
    dataset = datasaver.dataset

Starting experimental run with id: 6. 


# Converts Dataset Results to CSV

In [12]:
#Converts to pandas dataframe
df = dataset.to_pandas_dataframe()
#Converts dataframe to csv
df.to_csv(csvPath)

# Plot Dataset Results

In [ ]:
plot_dataset(dataset)